In [ ]:
import numpy as np
import random

In [ ]:
class AntColonyOptimization:
    def __init__(self, jobs, machines, processing_times, num_ants, num_iterations, alpha, beta, evaporation_rate, Q):
        self.jobs = jobs
        self.machines = machines
        self.processing_times = processing_times
        self.num_ants = num_ants
        self.num_iterations = num_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.Q = Q
        self.pheromone = np.ones((len(jobs), len(machines)))
        self.best_makespan = float('inf')
        self.best_solution = None

    def run(self):
        for iteration in range(self.num_iterations):
            solutions = []
            for ant in range(self.num_ants):
                solution = self.construct_solution()
                solutions.append(solution)
                self.update_pheromone(solution)
            self.global_update_pheromone(solutions)
            self.update_best_solution(solutions)

    def construct_solution(self):
      solution = []
      for job in self.jobs:
          job_sequence = self.processing_times[job]
          for machine, _ in job_sequence:
              solution.append((job, machine))
      return solution

    def select_machine(self, job, remaining_machines):
      job_sequence = self.processing_times[job]
      machine_index = next((i for i, (m, _) in enumerate(job_sequence) if m in remaining_machines), None)
      if machine_index is not None:
          return job_sequence[machine_index][0]
      else:
          return None

    def update_pheromone(self, solution):
      makespan = self.calculate_makespan(solution)
      for job, machine in solution:
          machine_index = next(i for i, (m, _) in enumerate(self.processing_times[job]) if m == machine)
          self.pheromone[self.jobs.index(job), machine_index] += self.Q / makespan

    def global_update_pheromone(self, solutions):
        self.pheromone = (1 - self.evaporation_rate) * self.pheromone
        for solution in solutions:
            self.update_pheromone(solution)

    def update_best_solution(self, solutions):
        for solution in solutions:
            makespan = self.calculate_makespan(solution)
            if makespan < self.best_makespan:
                self.best_makespan = makespan
                self.best_solution = solution

    def calculate_makespan(self, solution):
      job_end_times = {}
      machine_end_times = {machine: 0 for machine in self.machines}
      for job, machine in solution:
          processing_time = next(t for m, t in self.processing_times[job] if m == machine)
          start_time = max(job_end_times.get(job, 0), machine_end_times[machine])
          end_time = start_time + processing_time
          job_end_times[job] = end_time
          machine_end_times[machine] = end_time
      return max(job_end_times.values())

In [ ]:
# Example usage
jobs = ['J1', 'J2', 'J3']
machines = ['M1', 'M2', 'M3']
processing_times = {
    'J1': [('M3', 4), ('M2', 3), ('M1', 3)],
    'J2': [('M2', 1), ('M3', 2), ('M1', 4)],
    'J3': [('M2', 3), ('M1', 2), ('M3', 3)]
}

In [ ]:
num_ants = 30
num_iterations = 1000
alpha = 1
beta = 2
evaporation_rate = 0.5
Q = 100

aco = AntColonyOptimization(jobs, machines, processing_times, num_ants, num_iterations, alpha, beta, evaporation_rate, Q)
aco.run()

print("Best solution:", aco.best_solution)
print("Best makespan:", aco.best_makespan)

Best solution: [('J1', 'M3'), ('J1', 'M2'), ('J1', 'M1'), ('J2', 'M2'), ('J2', 'M3'), ('J2', 'M1'), ('J3', 'M2'), ('J3', 'M1'), ('J3', 'M3')]
Best makespan: 19
